In [1]:
import pandas as pd
import sys
import itertools

In [11]:
def analyze(sentences_json_filename, sampled_dest=None):
    df = pd.read_json(sentences_json_filename, lines=True)
    mapping = {}
    for _, row in df.iterrows():
        mentions = [x['text'] for x in row['entityMentions']]
        mentions = list(set(mentions))  # there can be multiple mentions of the same entity in the sentence
        if len(mentions) < 2:
            continue
        pairs = list(itertools.combinations(mentions, 2))
        sent = ' '.join(row['tokens'])
        for pair in pairs:
            pair_key = str(set(pair))
            sents = mapping.get(pair_key, [])
            sents.append(sent)
            mapping[pair_key] = sents

    print(len(mapping))
    lst = []
    for key, val in mapping.items():
        lst.append({"pair": key, "mentions": val, "mention_ct": len(val)})
    df = pd.DataFrame(lst)
    df = df.sort_values(by='mention_ct', ascending=False)
    if sampled_dest is not None:
        df.head(1000).to_csv(sampled_dest, index=None)
    return df

In [4]:
dataset = 'imdb'

In [5]:
embedding = 'meg-ac'

In [6]:
filename = '../../data/'+dataset+'-'+embedding+'/intermediate/sentences.json'

In [12]:
df =  analyze(filename,'autophrase_'+dataset+'.csv')

318112


In [8]:
df.head(200)

,pair,mentions,mention_ct
775,"{'movie', 'film'}",[His touch is most apparent during the film 's...,1423
48,"{'director', 'film'}",[ King criticized the director for changing m...,631
691,"{'film', 'films'}","[Unlike the dancing films in Hollywood , the c...",552
1332,"{'movie', 'life'}","[This movie portrays a difficult life ., that ...",511
123,"{'action', 'movie'}","[Good action sequences , plenty of "" feel good...",505
...,...,...,...
6821,"{'lead', 'film'}",[/>Busey may not have the physical presence of...,69
7484,"{'movie', 'directors'}","[This movie , is a must see if you want to kno...",68
2032,"{'vhs', 'dvd'}","[ DVD and VHS ., One thing does concern me th...",68
21139,"{'car', 'film'}",[To make it short and not to spoil everything ...,68
